# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Исследование надежности заёмщиков

<br>Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

<br>Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<br>В качестве исходных данных получена таблица с информацией о клиентах. 
<br>Поля таблицы:
<br>**children** — количество детей в семье
<br>**days_employed** — общий трудовой стаж в днях
<br>**dob_years** — возраст клиента в годах
<br>**education** — уровень образования клиента
<br>**education_id** — идентификатор уровня образования
<br>**family_status** — семейное положение
<br>**family_status_id** — идентификатор семейного положения
<br>**gender** — пол клиента
<br>**income_type** — тип занятости
<br>**debt** — имел ли задолженность по возврату кредитов
<br>**total_income** — ежемесячный доход
<br>**purpose** — цель получения кредита

Исследование разделено на несколько частей:

###### Данное исследование разделим на несколько частей.

##### Часть 1. Изучение общей информации:
* [1. Изучение файлов с данными, получение общей информации, загрузка библиотек.](#ch_1_1)

##### Часть 2. Предобработка данных:
* [1. Нахождение и ликвидация пропусков.](#ch_2_1)
* [2. Приведение данных к нужным типам.](#ch_2_2)
* [3. Удаление дубликатов.](#ch_2_3)
* [4. Лемматизация данных.](#ch_2_4)
* [5. Выделение категорий.](#ch_2_5)

##### Часть 3. Проведение анализа:
* [1. Исследование зависимости возврата кредита в срок от категории клиента.](#ch_3_1)

##### Часть 4. Выводы:
* [1. Выводы.](#ch_4_1)

## Шаг 1. Откройте файл с данными и изучите общую информацию<a id='ch_1_1'></a>

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pymystem3 import Mystem
m = Mystem()
df = pd.read_csv('/datasets/data.csv')
#print(df.head(10))
df.info()
#изменим наименования образований в нижний регистр
df.loc[:,'education'] = df.loc[:,'education'].str.lower()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Основной момент, бросающийся в глаза - отрицательные, но реалистичные показатели стажа работы у всех групп, кроме пенсионеров и безработных. У этих двух групп значения невозможно высокие.

## Шаг 2. Предобработка данных

### Обработка пропусков <a id='ch_2_1'></a>

In [2]:
print('До обработки: \n',df.isnull().sum())
#print(df.groupby('education')['total_income'].median())
#edu_medians = df.groupby('education')['total_income'].median()
median_income = df['total_income'].median()

#for row in range(len(df)):

    
avg_workdays_employed = df[(df['income_type'] != 'пенсионер') & (df['income_type'] != 'безработный')]['days_employed'].mean()
#avg_workdays_unemployed = df[(df['income_type'] == 'пенсионер') | (df['income_type'] == 'безработный')]['days_employed'].mean()
df['total_income'] = df['total_income'].fillna(median_income)
df['days_employed'] = df['days_employed'].fillna(avg_workdays_employed)
print('После обработки: \n',df.isnull().sum())


До обработки: 
 children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
После обработки: 
 children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод**

Пропущенные значения обнаружены в столбцах days_employed и total_income.
Пропуски являются полностью случайными и могли возникнуть в результате ошибки заполнения.
Значения заменены следующим образом: в total_income вместо пустых значений подставлено медианное значение дохода(из-за больших разбросов зарплат), в days employed - среднее значение стажа для устроенных на работу(т.к. для пенсионеров и безработных данные являются нереалистично завышенными).

# Замена типа данных <a id='ch_2_2'></a>

In [4]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

В DataFrame два столбца с вещественным типом данных: days_employed и total_imcome. В качестве метода перевода типа данных использован .astype().

### Обработка дубликатов <a id='ch_2_3'></a>

In [5]:
df.duplicated().sum()
df = df.drop_duplicates().reset_index(drop= True)
df.duplicated().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

В таблице наблюдается ~~54~~ 71 дубликат~~а~~. Найдены они с помощью метода .duplicated().sum(). Дубликаты удалены методом .drop_duplicates(). Индексы сброшены методом .reset_index(drop = True). Причиной появления подобных полных дубликатов может быть ошибка работы системы, при которой одна заявка оператора вносится в БД несколько раз, либо ошибка оператора, когда идентичные данные вносятся в БД несколько раз.

### Лемматизация <a id='ch_2_4'></a>

In [6]:
#lemmas = m.lemmatize(df['purpose'])
#print(df['purpose'].value_counts())
def lemm(purpose):
    return m.lemmatize(purpose)

df['purpose_lemmatized'] = df['purpose'].apply(lemm)
#print(df.head)

**Вывод**

Оптимальным способом при лемматизации кажется не проход циклом по строкам DataFrame, а написание функции лемматизации и применение ее методом apply к целевому столбцу.

### Категоризация данных <a id='ch_2_5'></a>

In [7]:
#берем значения по модулю для отрицательных величин стажа
df.loc[df['days_employed'] < 0, 'days_employed'] = df.loc[df['days_employed'] < 0, 'days_employed'] * (-1)
#берем значения по модулю для отрицательного количества детей
df.loc[df['children'] < 0, 'children'] = df.loc[df['children'] < 0, 'children'] * (-1)
#удаляем артефакт в поле с полом клиента
df = df.drop(df[df['gender'] == 'XNA'].index) 
#удаляем строки с возрастом 0 лет 
df = df.drop(df[df['dob_years'] == 0].index)

#df['dob_years'] = df[df['dob_years'] != 0] 
#print(df.groupby('income_type')['debt'].sum()/df.groupby('income_type')['debt'].count(), df.groupby('income_type')['debt'].count())
#print(df.groupby('family_status')['debt'].count())
def have_children(amount):
    if amount == 0:
        return 'Детей нет'
    return'Дети есть'

def credit_purpose(purpose):
    if any(i == 'автомобиль' for i in purpose):
        return 'автомобиль'
    elif any(i == 'свадьба' for i in purpose):
        return 'свадьба'
    elif any(i == 'образование' for i in purpose):
        return 'образование'
    elif any(i == 'недвижимость' or i == 'жилье' for i in purpose):
        return 'недвижимость'
    return 'другое'

def income_amount(income):
    if income < 50000:
        return 'до 50 тысяч рублей'
    elif 50000 <= income < 75000:
        return '50-75 тысяч рублей'
    elif 75000 <= income < 100000:
        return '75-100 тысяч рублей'
    elif 100000 <= income < 150000:
        return '100-150 тысяч рублей'
    elif 1500000 <= income < 200000:
        return '150-200 тысяч рублей'
    return 'более 200 тысяч рублей'
#добавление столбцов с группировками по наличию детей, типу кредита, доходу
df['have_childen'] = df['children'].apply(have_children)
df['purpose_grouped'] = df['purpose_lemmatized'].apply(credit_purpose)
df['income_grouped'] = df['total_income'].apply(income_amount)

#словарь для кодов образования
education_dict = df[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
#print(education_dict.sort_values('education_id', ascending = True))
#словарь для семейного статуса
family_stat_dict = df[['family_status', 'family_status_id']]
family_stat_dict['family_status'] = family_stat_dict['family_status'].str.lower()
family_stat_dict = family_stat_dict.drop_duplicates().reset_index(drop=True)
#print(family_stat_dict.sort_values('family_status_id', ascending = True))

#print(df.head(10))

**Вывод**

Данные очищены от артефактов (XNA в колонке gender, нулевой возраст), выведены категории по признаку наличия детей, цели кредита (исходя их имеющихся лемм), уровню дохода. Заведены словари для типов образования (предварительно приведены к нижнему регистру) и семейного статуса.

## Шаг 3. Ответьте на вопросы <a id='ch_3_1'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
#answ_1 = df.groupby('have_childen')['debt'].sum()/df.groupby('have_childen')['debt'].count()
#print(answ_1)
#print('Среди клиентов с детьми должников больше в {:.3f}'.format(answ_1[1]/answ_1[0]), 'раза')
pivot_1 = df.pivot_table(index='have_childen',  values = 'debt', aggfunc = ['sum', 'count'])
pivot_1['debt_rate'] = (pivot_1['sum'] / pivot_1['count'])#.apply('{:.2%}'.format)
pivot_1['debt_rate'] = pivot_1['debt_rate'].apply('{:.2%}'.format)
print(pivot_1)

               sum  count debt_rate
              debt   debt          
have_childen                       
Детей нет     1058  14021     7.55%
Дети есть      675   7331     9.21%


**Вывод**

Зависимость есть: среди клиентов с детьми на 2,21% больше должников.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
#answ_2 = df.groupby('family_status')['debt'].sum()/df.groupby('family_status')['debt'].count()
#print(answ_2.sort_values(ascending=True))
pivot_2 = df.pivot_table(index='family_status',  values = 'debt', aggfunc = ['sum', 'count'])
pivot_2['debt_rate'] = (pivot_2['sum'] / pivot_2['count'] * 100)
print(pivot_2)

                       sum  count debt_rate
                      debt   debt          
family_status                              
Не женат / не замужем  273   2794  9.770938
в разводе               85   1185  7.172996
вдовец / вдова          62    954  6.498952
гражданский брак       386   4129  9.348511
женат / замужем        927  12290  7.542718


**Вывод**

Если рассматривать семейные положения "гражданский брак" и "не женат/ не замужем" как отдельные сущности, то зависимость имеется, у этих двух категорий относительное количество должников выше. Однако, имеет смысл учитывать категорию "не женат/не замужем" вместе с "в разводе", и "гражданский брак" вместе с "женат/замужем" ввиду их относительной смысловой близости. В таком случае зависимость не наблюдается.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
#answ_3 = df.groupby('income_grouped')['debt'].sum()/df.groupby('income_grouped')['debt'].count()
#print(answ_3.sort_values(ascending=True))
pivot_3 = df.pivot_table(index='income_grouped',  values = 'debt', aggfunc = ['sum', 'count'])
pivot_3['debt_rate'] = (pivot_3['sum'] / pivot_3['count'] * 100)
print(pivot_3)

                        sum count debt_rate
                       debt  debt          
income_grouped                             
100-150 тысяч рублей    658  7775  8.463023
50-75 тысяч рублей      113  1486  7.604307
75-100 тысяч рублей     217  2584  8.397833
более 200 тысяч рублей  722  9137  7.901937
до 50 тысяч рублей       23   370  6.216216


**Вывод**

Зависимость имеется. Парадоксально, но количество должников увеличивается с увеличением роста доходов.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
#answ_4 = df.groupby('purpose_grouped')['debt'].sum()/df.groupby('purpose_grouped')['debt'].count()
#print(answ_4.sort_values(ascending=True))
pivot_4 = df.pivot_table(index='purpose_grouped',  values = 'debt', aggfunc = ['sum', 'count'])
pivot_4['debt_rate'] = (pivot_4['sum'] / pivot_4['count'] * 100)
print(pivot_4)

                 sum  count debt_rate
                debt   debt          
purpose_grouped                      
автомобиль       400   4284  9.337068
недвижимость     779  10763  7.237759
образование      370   3995  9.261577
свадьба          184   2310  7.965368


**Вывод**

Возврат в срок зависит от цели кредита: чаще всего долги имеются по кредитам на авто и образование. Реже всего просрочка имеется по кредитам на недвижимость.

## Шаг 4. Общий вывод<a id='ch_4_1'></a>

Проведя исследование, пришли к выводу, что наличие детей негативно влияет на факт погашения кредита в срок. Однозначной зависимости вероятности выхода на просрочку от семейного положения не найдено, т.к. есть группы семейных положений, имеющие отличающиеся показатели, но пересекающиеся в смысловом плане(например, женатые и состоящие в гражданском браке). Интересным также является то, что с ростом доходов клиенты реже гасят кредит без долгов. Наименее надежными для банка являются кредиты на образование и автомобиль - по ним должников больше всего.